In [ ]:
# Machine Learning Stuff
# X = Matrix, each row would be game, each column would be different feature (just drop columns I don't want in), y = Road Win
# Logistic Regression Model would be preferrable
# Train Test Split
# Use GridSearchCV, 5 or 10

In [1]:
# Importing necessary packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

In [2]:
# Uploading DataFrame and inspecting its contents
df = pd.DataFrame()
df = pd.read_csv('/Users/dwreiter/Desktop/Work/Springboard/NFL Capstone Project/Data Wrangling/nfl_df.csv', delimiter='\t', index_col=0)
df.head()

,Date,Home Field Goal Attempts,Home First Downs,Home Fourth Down Attempts,Home Fourth Down Successes,Home Fumble TDs,Home Fumbles,Home Fumbles Lost,Home Fumbles Recovered,Home Goal To Go Attempts,...,Road Time of Possession,Road Total TDs,Road Total Yds,Road Touchbacks,Road Two Point Conversion Attempts,Road Two Point Conversion Successes,Road Win,Road Wins,Season,Week Number
0,"on December 20, 2015",1,19,2,0,0,2,2,0,0,...,2248,5,506,3,0,0,True,1,nfl-2015-2016,15
1,"on November 29, 2015",2,17,2,1,0,2,1,0,0,...,2247,2,350,1,0,0,True,1,nfl-2015-2016,12
2,"on December 27, 2015",1,19,0,0,2,2,1,3,3,...,1784,1,265,0,1,1,False,0,nfl-2015-2016,16
3,"on November 22, 2015",2,21,0,0,0,2,0,1,1,...,1875,4,415,3,0,0,False,0,nfl-2015-2016,11
4,"on January 3, 2016",1,16,1,0,0,0,0,0,0,...,2197,4,382,7,0,0,True,1,nfl-2015-2016,17


In [3]:
# Setting X and y for Road Teams with X having just statistics, y being Road Win
RW_X = df.drop(['Season', 'Week Number', 'Date', 'Name of Winning Team', 'Name of Losing Team', 'Name of Tying Teams', 'Road Team Name', 'Home Team Name', 'Road Win', 'Road Loss', 'Road Tie', 'Road Wins', 'Road Losses', 'Road Ties', 'Home Win', 'Home Loss', 'Home Tie', 'Home Wins', 'Home Losses', 'Home Ties'], axis=1).values
RW_y = df['Road Win'].values

# Splitting the data into Train and Test
RW_X_train, RW_X_test, RW_y_train, RW_y_test = train_test_split(RW_X, RW_y, test_size = 0.2, random_state=42, stratify=RW_y)

In [4]:
# Setting up parameter C
c_space = np.logspace(-5, 8, 15)
RW_param_grid = {'C': c_space}

# Setting up Logistic Regression Model
RW_logreg = LogisticRegression()

# Using GridSearchCV to help avoid overfitting
RW_CV = GridSearchCV(RW_logreg, RW_param_grid, cv=5)

In [5]:
# Fitting Training Data to model and finding best parameter as well as score
RW_CV.fit(RW_X_train, RW_y_train)
print('Best Parameter for Road Win Model: ' + str(RW_CV.best_params_))
print('Best Score for Road Win Model: ' + str(RW_CV.best_score_))

Best Parameter for Road Win Model: {'C': 0.051794746792312128}
Best Score for Road Win Model: 0.974452554745


In [6]:
# Setting up prediction based on testing data for X
RW_y_pred = RW_CV.predict(RW_X_test)

# Printing R Squared
print('R squared: ' + str(RW_CV.score(RW_X_test, RW_y_test)))

# Computing and printing Means Squared Error
RW_rmse = np.sqrt(mean_squared_error(RW_y_test, RW_y_pred))
print('Root Mean Squared Error: ' + str(RW_rmse))

R squared: 0.978260869565
Root Mean Squared Error: 0.147441956155


/Users/dwreiter/miniconda2/envs/tutorial/lib/python2.7/site-packages/sklearn/metrics/regression.py:239: DeprecationWarning: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,


In [10]:
# Setting X and y for Home Teams with X having just statistics, y being Home Win
HW_X = df.drop(['Season', 'Week Number', 'Date', 'Name of Winning Team', 'Name of Losing Team', 'Name of Tying Teams', 'Road Team Name', 'Home Team Name', 'Road Win', 'Road Loss', 'Road Tie', 'Road Wins', 'Road Losses', 'Road Ties', 'Home Win', 'Home Loss', 'Home Tie', 'Home Wins', 'Home Losses', 'Home Ties'], axis=1).values
HW_y = df['Home Win'].values

# Splitting the data into Train and Test
HW_X_train, HW_X_test, HW_y_train, HW_y_test = train_test_split(HW_X, HW_y, test_size = 0.2, random_state=42, stratify=HW_y)

In [11]:
# Setting up parameter C
c_space = np.logspace(-5, 8, 15)
HW_param_grid = {'C': c_space}

# Setting up Logistic Regression Model
HW_logreg = LogisticRegression()

# Using GridSearchCV to help avoid overfitting
HW_CV = GridSearchCV(HW_logreg, HW_param_grid, cv=5)

In [12]:
# Fitting Training Data to model and finding best parameter as well as score
HW_CV.fit(HW_X_train, HW_y_train)
print('Best Parameter for Home Win Model: ' + str(HW_CV.best_params_))
print('Best Score for Home Win Model: ' + str(HW_CV.best_score_))

Best Parameter for Home Win Model: {'C': 19306.977288832535}
Best Score for Home Win Model: 0.978102189781


In [13]:
# Setting up prediction based on testing data for X
HW_y_pred = HW_CV.predict(HW_X_test)

# Printing R Squared
print('R squared: ' + str(HW_CV.score(HW_X_test, HW_y_test)))

# Computing and printing Means Squared Error
HW_rmse = np.sqrt(mean_squared_error(HW_y_test, HW_y_pred))
print('Root Mean Squared Error: ' + str(HW_rmse))

R squared: 0.985507246377
Root Mean Squared Error: 0.120385853086


/Users/dwreiter/miniconda2/envs/tutorial/lib/python2.7/site-packages/sklearn/metrics/regression.py:239: DeprecationWarning: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,
